# TACH1

This Notebook contains the following: 

    querying GraphDB
    
    cleaning data
    
    Parametric queries testing
    
    Nodes visualization
    
    REST API tests 

In [2]:
import rdflib
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
import networkx as nx
import matplotlib.pyplot as plt
import re
from random import sample
import graphdb
import pandas as pd
import numpy as np

from SPARQLWrapper import SPARQLWrapper, JSON

from pyvis.network import Network
import random

import requests

import urllib.request

import urllib3

In [30]:
sparql = SPARQLWrapper(
    "http://LAPTOP-CS19OKIS:7200/repositories/tach1"
)
sparql.setReturnFormat(JSON)

queryString4="""
SELECT ?o ?p ?s WHERE{
    ?o ?p ?s
    }
"""

In [31]:
sparql.setQuery(queryString4)

test=[]
dataf={}
try:
    ret = sparql.queryAndConvert()
    dataf['o']=[]
    dataf['p']=[]
    dataf['s']=[]
    for r in ret["results"]["bindings"]:
        #print(r)
        #print(r["O"]["value"])
        dataf['o'].append(r['o']["value"])
        dataf['p'].append(r['p']["value"])
        dataf['s'].append(r['s']["value"])
        if r['o']["value"] not in test:
            test.append(r['o']["value"])
        if r["s"]["value"] not in test:    
            test.append(r["s"]["value"])
    

        
except Exception as e:
    print(e)


df=pd.DataFrame(dataf)

In [32]:
def cleanDataFrame(df):

    
    dataset=df
    
    listeu=dataset["p"]
    dataset=dataset[-dataset["o"].isin(listeu)]
    
    dataset=dataset[-dataset["o"].str.match(".*www.w3.org.*")]
    

    
    pat = r'.*\/(.)'
    dataset["s"]= dataset["s"].str.replace(pat,r'\1',regex=True)
    dataset["o"]= dataset["o"].str.replace(pat,r'\1',regex=True)
    

    
    dataset["o"]=dataset["o"].str.replace(".*#(.)",r'\1',regex=True)
    dataset["s"]=dataset["s"].str.replace(".*#(.)",r'\1',regex=True)
    
    lab=dataset[dataset["p"].isin(["http://www.w3.org/2000/01/rdf-schema#label","http://www.w3.org/2006/vcard/ns#fn"])]

    
    dataset=pd.concat([lab, dataset]).drop_duplicates(keep=False)
    labels=lab["s"].tolist()
    urls=lab["o"].tolist()
    dataset=dataset.replace(urls,labels)

    return dataset

momo=cleanDataFrame(df)
momo

,o,p,s
33,Aín,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,Municipio
37,BENIALI,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,hotel
41,Comunidad Valenciana,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,ComunidadAutonoma
43,Plana_Baja,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,Comarca
45,provincia de Castellón,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,Provincia
...,...,...,...
7562,LA BOLERA,http://www.w3.org/2006/vcard/ns#hasEmail,hotelrestaurantelabolera@hotmail.com
7563,CRYSTAL-PARK,http://www.w3.org/2006/vcard/ns#hasEmail,hotelcrystalpark@gmail.com
7564,NOU CASABLANCA,http://www.w3.org/2006/vcard/ns#hasEmail,noucasablanca@gmail.com
7565,RH VINAROS PLAYA,http://www.w3.org/2006/vcard/ns#hasEmail,vinarosplaya@hotelesrh.com


# testing the content of the dataframe

In [33]:
momo[momo["o"].str.match("node742")]
momo[momo["o"].str.match("A.n") | momo["s"].str.match("A.n")]

,o,p,s
33,Aín,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,Municipio
1203,Aín,http://www.w3.org/2002/07/owl#sameAs,Aín
1204,Aín,http://www.w3.org/2002/07/owl#sameAs,Q735833
2406,Aín,http://krono.act.uji.es/schemas/slodbi#num-apa...,1
2511,Aín,http://krono.act.uji.es/schemas/slodbi#plazas-...,8
2616,Aín,http://krono.act.uji.es/schemas/slodbi#tiene-h...,BENIALI
2786,Aín,http://krono.act.uji.es/schemas/slodbi#wd-entity,Q735833
3235,Aín,http://krono.act.uji.es/vocabulary/slodbi#hasProv,dipcas-dataset
3370,Aín,http://vocab.linkeddata.es/datosabiertos/def/s...,Comunidad Valenciana
3514,Aín,http://vocab.linkeddata.es/datosabiertos/def/s...,Plana_Baja


In [34]:
df[df["o"].str.match("node742")] #original
df[df["o"].str.match(".*A.n.*")] #original
df[df["o"].str.match(".*A.n.*") | momo["s"].str.match(".*A.n.*")]

,o,p,s
33,http://krono.act.uji.es/schemas/slodbi#Ain,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://vocab.linkeddata.es/datosabiertos/def/s...
1203,http://krono.act.uji.es/schemas/slodbi#Ain,http://www.w3.org/2002/07/owl#sameAs,http://krono.act.uji.es/schemas/slodbi#Ain
1204,http://krono.act.uji.es/schemas/slodbi#Ain,http://www.w3.org/2002/07/owl#sameAs,http://www.wikidata.org/entity/Q735833
1966,http://krono.act.uji.es/schemas/slodbi#Ain,http://www.w3.org/2000/01/rdf-schema#label,Aín
2406,http://krono.act.uji.es/schemas/slodbi#Ain,http://krono.act.uji.es/schemas/slodbi#num-apa...,1
2511,http://krono.act.uji.es/schemas/slodbi#Ain,http://krono.act.uji.es/schemas/slodbi#plazas-...,8
2616,http://krono.act.uji.es/schemas/slodbi#Ain,http://krono.act.uji.es/schemas/slodbi#tiene-h...,http://krono.act.uji.es/schemas/slodbi#hotel-7...
2786,http://krono.act.uji.es/schemas/slodbi#Ain,http://krono.act.uji.es/schemas/slodbi#wd-entity,http://www.wikidata.org/entity/Q735833
3235,http://krono.act.uji.es/schemas/slodbi#Ain,http://krono.act.uji.es/vocabulary/slodbi#hasProv,http://krono.act.uji.es/vocabulary/slodbi#dipc...
3370,http://krono.act.uji.es/schemas/slodbi#Ain,http://vocab.linkeddata.es/datosabiertos/def/s...,http://krono.act.uji.es/schemas/slodbi#Comunid...


<h1> STEP BY STEP PROCESS </h1>

# Supression of predicate nodes

In [35]:
listeu=df["p"]
print(listeu)
df1=df[-df["o"].isin(listeu)]
df1

0       http://www.w3.org/1999/02/22-rdf-syntax-ns#type
1       http://www.w3.org/1999/02/22-rdf-syntax-ns#type
2       http://www.w3.org/1999/02/22-rdf-syntax-ns#type
3       http://www.w3.org/1999/02/22-rdf-syntax-ns#type
4       http://www.w3.org/1999/02/22-rdf-syntax-ns#type
                             ...                       
7562           http://www.w3.org/2006/vcard/ns#hasEmail
7563           http://www.w3.org/2006/vcard/ns#hasEmail
7564           http://www.w3.org/2006/vcard/ns#hasEmail
7565           http://www.w3.org/2006/vcard/ns#hasEmail
7566           http://www.w3.org/2006/vcard/ns#hasEmail
Name: p, Length: 7567, dtype: object


,o,p,s
8,http://www.w3.org/2002/07/owl#equivalentProperty,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#SymmetricProperty
9,http://www.w3.org/2002/07/owl#equivalentProperty,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#TransitiveProperty
10,http://www.w3.org/2002/07/owl#equivalentClass,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#SymmetricProperty
11,http://www.w3.org/2002/07/owl#equivalentClass,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#TransitiveProperty
15,http://www.w3.org/1999/02/22-rdf-syntax-ns#sub...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...
...,...,...,...
7562,http://krono.act.uji.es/schemas/slodbi#hotel-7...,http://www.w3.org/2006/vcard/ns#hasEmail,hotelrestaurantelabolera@hotmail.com
7563,http://krono.act.uji.es/schemas/slodbi#hotel-8...,http://www.w3.org/2006/vcard/ns#hasEmail,hotelcrystalpark@gmail.com
7564,http://krono.act.uji.es/schemas/slodbi#hotel-9...,http://www.w3.org/2006/vcard/ns#hasEmail,noucasablanca@gmail.com
7565,http://krono.act.uji.es/schemas/slodbi#hotel-9...,http://www.w3.org/2006/vcard/ns#hasEmail,vinarosplaya@hotelesrh.com


In [36]:
df[df["o"].isin(df1["p"].tolist())]

,o,p,s
0,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...
1,http://www.w3.org/2000/01/rdf-schema#subProper...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...
2,http://www.w3.org/2000/01/rdf-schema#subProper...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#TransitiveProperty
3,http://www.w3.org/2000/01/rdf-schema#subClassOf,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...
4,http://www.w3.org/2000/01/rdf-schema#subClassOf,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#TransitiveProperty
...,...,...,...
1202,http://www.w3.org/2000/01/rdf-schema#label,http://www.w3.org/2000/01/rdf-schema#range,http://www.w3.org/2000/01/rdf-schema#Literal
1957,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://proton.semanticweb.org/protonsys#transi...,http://www.w3.org/2000/01/rdf-schema#subClassOf
1958,http://www.w3.org/2000/01/rdf-schema#subProper...,http://proton.semanticweb.org/protonsys#transi...,http://www.w3.org/2000/01/rdf-schema#subProper...
1959,http://www.w3.org/2000/01/rdf-schema#subClassOf,http://proton.semanticweb.org/protonsys#transi...,http://www.w3.org/2000/01/rdf-schema#subClassOf


In [37]:
#verification1
df1[df1["o"].isin(df1["p"].tolist())]

,o,p,s


In [38]:
#verification2
df1[df1["o"].str.match(".*www.w3")].head()

,o,p,s
8,http://www.w3.org/2002/07/owl#equivalentProperty,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#SymmetricProperty
9,http://www.w3.org/2002/07/owl#equivalentProperty,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#TransitiveProperty
10,http://www.w3.org/2002/07/owl#equivalentClass,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#SymmetricProperty
11,http://www.w3.org/2002/07/owl#equivalentClass,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#TransitiveProperty
15,http://www.w3.org/1999/02/22-rdf-syntax-ns#sub...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...


In [39]:
#rectification: supression of the w3 nodes
df1=df1[-df1["o"].str.match(".*www.w3.org.*")]
df1

,o,p,s
33,http://krono.act.uji.es/schemas/slodbi#Ain,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://vocab.linkeddata.es/datosabiertos/def/s...
37,http://krono.act.uji.es/schemas/slodbi#hotel-7...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://krono.act.uji.es/schemas/slodbi#hotel
41,http://krono.act.uji.es/schemas/slodbi#Comunid...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://vocab.linkeddata.es/datosabiertos/def/s...
43,http://krono.act.uji.es/schemas/slodbi#Plana_Baja,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://vocab.linkeddata.es/datosabiertos/def/s...
45,http://krono.act.uji.es/schemas/slodbi#Castellon,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://vocab.linkeddata.es/datosabiertos/def/s...
...,...,...,...
7562,http://krono.act.uji.es/schemas/slodbi#hotel-7...,http://www.w3.org/2006/vcard/ns#hasEmail,hotelrestaurantelabolera@hotmail.com
7563,http://krono.act.uji.es/schemas/slodbi#hotel-8...,http://www.w3.org/2006/vcard/ns#hasEmail,hotelcrystalpark@gmail.com
7564,http://krono.act.uji.es/schemas/slodbi#hotel-9...,http://www.w3.org/2006/vcard/ns#hasEmail,noucasablanca@gmail.com
7565,http://krono.act.uji.es/schemas/slodbi#hotel-9...,http://www.w3.org/2006/vcard/ns#hasEmail,vinarosplaya@hotelesrh.com


# url cleaning

In [40]:
pat = r'.*\/(.)'
df1["s"]= df1["s"].str.replace(pat,r'\1',regex=True)
df1["o"]= df1["o"].str.replace(pat,r'\1',regex=True)
df1

C:\Users\gmati\AppData\Local\Temp\ipykernel_32644\126304514.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["s"]= df1["s"].str.replace(pat,r'\1',regex=True)
C:\Users\gmati\AppData\Local\Temp\ipykernel_32644\126304514.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["o"]= df1["o"].str.replace(pat,r'\1',regex=True)


,o,p,s
33,slodbi#Ain,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,territorio#Municipio
37,slodbi#hotel-7a2575c0543108e27c8ebf3cdf705d98e...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,slodbi#hotel
41,slodbi#Comunidad_Valenciana,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,territorio#ComunidadAutonoma
43,slodbi#Plana_Baja,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,territorio#Comarca
45,slodbi#Castellon,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,territorio#Provincia
...,...,...,...
7562,slodbi#hotel-794e9b67799c1affdc9cacce8439a9b53...,http://www.w3.org/2006/vcard/ns#hasEmail,hotelrestaurantelabolera@hotmail.com
7563,slodbi#hotel-80b917fa788e75b5e8de2102d32fb80da...,http://www.w3.org/2006/vcard/ns#hasEmail,hotelcrystalpark@gmail.com
7564,slodbi#hotel-97c90348c8f61ecf6b5a326af9a025d12...,http://www.w3.org/2006/vcard/ns#hasEmail,noucasablanca@gmail.com
7565,slodbi#hotel-9aa7d04691c8b162dc59de7bc00038d6a...,http://www.w3.org/2006/vcard/ns#hasEmail,vinarosplaya@hotelesrh.com


In [41]:
df[df["o"].str.match(".*albergue.*")]

,o,p,s
76,http://krono.act.uji.es/schemas/slodbi#tiene-a...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...
77,http://krono.act.uji.es/schemas/slodbi#albergu...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://krono.act.uji.es/schemas/slodbi#albergue
86,http://krono.act.uji.es/schemas/slodbi#albergu...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://krono.act.uji.es/schemas/slodbi#albergue
216,http://krono.act.uji.es/schemas/slodbi#albergu...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://krono.act.uji.es/schemas/slodbi#albergue
221,http://krono.act.uji.es/schemas/slodbi#albergu...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://krono.act.uji.es/schemas/slodbi#albergue
...,...,...,...
6988,http://krono.act.uji.es/schemas/slodbi#albergu...,http://www.w3.org/2006/vcard/ns#url,http://vallibona.es
6989,http://krono.act.uji.es/schemas/slodbi#albergu...,http://www.w3.org/2006/vcard/ns#url,http://www.vilafames.es
6990,http://krono.act.uji.es/schemas/slodbi#albergu...,http://www.w3.org/2006/vcard/ns#url,http://www.laparreta.com
6992,http://krono.act.uji.es/schemas/slodbi#albergu...,http://www.w3.org/2006/vcard/ns#url,http://www.elmasdeborras.com


In [42]:
lab=df1[df1["p"].isin(["http://www.w3.org/2000/01/rdf-schema#label","http://www.w3.org/2006/vcard/ns#fn"])]

labels=lab["s"].tolist()
urls=lab["o"].tolist()

lab

,o,p,s
1966,slodbi#Ain,http://www.w3.org/2000/01/rdf-schema#label,Aín
1967,slodbi#Comunidad_Valenciana,http://www.w3.org/2000/01/rdf-schema#label,Comunidad Valenciana
1968,slodbi#Castellon,http://www.w3.org/2000/01/rdf-schema#label,provincia de Castellón
1969,slodbi#Albocasser,http://www.w3.org/2000/01/rdf-schema#label,Albocàsser
1970,slodbi#Alcala_de_Xivert,http://www.w3.org/2000/01/rdf-schema#label,Alcalà de Xivert
...,...,...,...
4837,slodbi#empresa-turismo-activo-b8e797b9324c5126...,http://www.w3.org/2006/vcard/ns#fn,PERSENDA ESPORTS D'AVENTURA
4838,slodbi#hotel-a6fce53c479eea7ad373abbe733843b85...,http://www.w3.org/2006/vcard/ns#fn,VILLA DE VIVER
4839,slodbi#empresa-turismo-activo-5ffe966fb623d602...,http://www.w3.org/2006/vcard/ns#fn,TRACKS MAESTRAT
4840,slodbi#hotel-ce4822401290083e77640dc6797745a97...,http://www.w3.org/2006/vcard/ns#fn,HOSPEDERIA LA BALMA


In [43]:
dup=lab[lab["s"].duplicated()]
lab[lab["s"].isin(dup["s"])]
#EL FAIXERO
df1[df1["o"].isin(df1[df1["s"]=="EL FAIXERO"]["o"])]
#node72 TELEPHONE
#node872 TELEPHONE
#node70 ADRESS
#node870 ADRESSS

df[df["o"].isin(["node872","node72","node70","node870"])]
#two different adresses yet same hotel name
#same phone number

# they need to be called differently 

,o,p,s
551,node72,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2006/vcard/ns#Voice
960,node872,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2006/vcard/ns#Voice
5163,node70,http://krono.act.uji.es/schemas/slodbi#locality,node71
5363,node870,http://krono.act.uji.es/schemas/slodbi#locality,node871
5467,node70,http://www.w3.org/2006/vcard/ns#locality,Cinctorres
5667,node870,http://www.w3.org/2006/vcard/ns#locality,Cinctorres
5771,node70,http://www.w3.org/2006/vcard/ns#postal-code,12318
5947,node870,http://www.w3.org/2006/vcard/ns#postal-code,12318
6031,node70,http://www.w3.org/2006/vcard/ns#street-address,CR LA IGLESUELA 7
6231,node870,http://www.w3.org/2006/vcard/ns#street-address,C SAN LUIS 59-63


In [44]:
#dup=lab[lab["s"].duplicated()]
lab[lab["s"].isin(dup["s"])]


,o,p,s
4599,slodbi#hotel-b6b3d402bdccd6f9464f63db9242b2089...,http://www.w3.org/2006/vcard/ns#fn,VORAMAR
4653,slodbi#hotel-ea875070569331701d1e8543d919404d4...,http://www.w3.org/2006/vcard/ns#fn,JAIME I
4656,slodbi#albergue-b4619d5f34e51da488f9fb19ebb8e8...,http://www.w3.org/2006/vcard/ns#fn,EL FAIXERO
4658,slodbi#hotel-a377b0efafad52f870ecb53baf99ee4bd...,http://www.w3.org/2006/vcard/ns#fn,EL FAIXERO
4675,slodbi#albergue-301edb33dc7276e7346e6cab110788...,http://www.w3.org/2006/vcard/ns#fn,ALBERGUE EL DHARMA
4676,slodbi#albergue-82bfe9a537b6f67b814b617d7949b4...,http://www.w3.org/2006/vcard/ns#fn,ALBERGUE EL DHARMA
4690,slodbi#hotel-feeaceefcc18b61940ef99a53fa5401d9...,http://www.w3.org/2006/vcard/ns#fn,EL CID
4703,slodbi#hotel-6bb2e7219ed7a10e17133e9cff88b93dd...,http://www.w3.org/2006/vcard/ns#fn,VORAMAR
4713,slodbi#hotel-1fc077aa276d36d3b717d7c9462d2287c...,http://www.w3.org/2006/vcard/ns#fn,MARINA
4724,slodbi#hotel-03c12f74a0d889d106ab42f730b2375a3...,http://www.w3.org/2006/vcard/ns#fn,EL CID


In [45]:
df2=pd.concat([lab, df1]).drop_duplicates(keep=False)





In [46]:
newlabels=[]
tempo={}
for lab in labels:
    if lab in tempo:
        tempo[lab]=tempo[lab]+1
        newlabels.append(lab+'('+str(tempo[lab])+')')
    else:
        tempo[lab]=0
        newlabels.append(lab)


In [47]:
df3=df2.replace(urls,newlabels)
df3[df3["o"].str.match(".*JAIME.*")]

,o,p,s
212,JAIME I,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,slodbi#hotel
282,REY DON JAIME,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,slodbi#hotel
346,JAIME I(1),http://www.w3.org/1999/02/22-rdf-syntax-ns#type,slodbi#hotel
1366,JAIME I,http://www.w3.org/2002/07/owl#sameAs,ea875070569331701d1e8543d919404d4f93cb3c/
1436,REY DON JAIME,http://www.w3.org/2002/07/owl#sameAs,f8ee71c19e02085c5b57ed5843356be44ca7c26f/
1500,JAIME I(1),http://www.w3.org/2002/07/owl#sameAs,4a5803404660f9b81311404a6b668eb787f04323/
4086,JAIME I,http://krono.act.uji.es/schemas/slodbi#categor...,3
4110,REY DON JAIME,http://krono.act.uji.es/schemas/slodbi#categor...,3
4152,JAIME I(1),http://krono.act.uji.es/schemas/slodbi#categor...,3
4350,JAIME I,http://krono.act.uji.es/schemas/slodbi#ubicacion,node1128


In [48]:
print(df2[df2["s"]=="node473"])
print("-------")
print(df2[df2["o"]=="node473"])


                                                      o  \
4410  slodbi#hotel-1fc077aa276d36d3b717d7c9462d2287c...   

                                                     p        s  
4410  http://krono.act.uji.es/schemas/slodbi#ubicacion  node473  
-------
            o                                                p  \
761   node473  http://www.w3.org/1999/02/22-rdf-syntax-ns#type   
2221  node473       http://www.opengis.net/ont/geosparql#asWKT   

                                              s  
761                                    sf#Point  
2221  POINT(40.09360759100008 0.12244000000004)  


In [49]:
tab=df3[df3["s"]=="territorio#Comarca"]
coma=tab["o"]
coma=coma.tolist()

df3[df3["o"].isin(coma)]

,o,p,s
43,slodbi#Plana_Baja,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,territorio#Comarca
47,slodbi#Alto_Maestrazgo,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,territorio#Comarca
70,slodbi#Bajo_Maestrazgo,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,territorio#Comarca
73,slodbi#Alcalaten,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,territorio#Comarca
81,slodbi#Alto_Palancia,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,territorio#Comarca
84,slodbi#Plana_Alta,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,territorio#Comarca
95,slodbi#Alto_Mijares,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,territorio#Comarca
178,slodbi#Los_Puertos_de_Morella,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,territorio#Comarca
1207,slodbi#Plana_Baja,http://www.w3.org/2002/07/owl#sameAs,Q2300704
1210,slodbi#Alto_Maestrazgo,http://www.w3.org/2002/07/owl#sameAs,Q2247639


In [50]:
df3[df3["o"].str.match(".*CIRCUITO.*")]


,o,p,s
277,CIRCUITO DE AVENTURA SALTAPINS,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,slodbi#empresa-turismo-activo
1431,CIRCUITO DE AVENTURA SALTAPINS,http://www.w3.org/2002/07/owl#sameAs,12cc2298d04078ad585ffe4287aec35dd12693cc/
4381,CIRCUITO DE AVENTURA SALTAPINS,http://krono.act.uji.es/schemas/slodbi#ubicacion,node109
4988,CIRCUITO DE AVENTURA SALTAPINS,http://www.w3.org/2006/vcard/ns#hasAddress,node110
6464,CIRCUITO DE AVENTURA SALTAPINS,http://www.w3.org/2006/vcard/ns#hasTelephone,node112
6967,CIRCUITO DE AVENTURA SALTAPINS,http://www.w3.org/2006/vcard/ns#url,saltapins.com


In [51]:
df3["o"]=df3["o"].str.replace(".*#(.)",r'\1',regex=True)
df3["s"]=df3["s"].str.replace(".*#(.)",r'\1',regex=True)
df3

,o,p,s
33,Aín,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,Municipio
37,BENIALI,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,hotel
41,Comunidad Valenciana,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,ComunidadAutonoma
43,Plana_Baja,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,Comarca
45,provincia de Castellón,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,Provincia
...,...,...,...
7562,LA BOLERA,http://www.w3.org/2006/vcard/ns#hasEmail,hotelrestaurantelabolera@hotmail.com
7563,CRYSTAL-PARK,http://www.w3.org/2006/vcard/ns#hasEmail,hotelcrystalpark@gmail.com
7564,NOU CASABLANCA,http://www.w3.org/2006/vcard/ns#hasEmail,noucasablanca@gmail.com
7565,RH VINAROS PLAYA,http://www.w3.org/2006/vcard/ns#hasEmail,vinarosplaya@hotelesrh.com


In [52]:
df3[df3.duplicated()]

,o,p,s


In [53]:
df3[df3["o"].str.match(".*MAS DE BORRAS.*")]

,o,p,s
466,MAS DE BORRAS,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,albergue
468,MAS DE BORRAS(1),http://www.w3.org/1999/02/22-rdf-syntax-ns#type,empresa-turismo-activo
1620,MAS DE BORRAS,http://www.w3.org/2002/07/owl#sameAs,3a2898bf67bc27a220580061867d977f9b17c708/
1622,MAS DE BORRAS(1),http://www.w3.org/2002/07/owl#sameAs,9fa5950eb00ceb3f4a6d7c013dff0ce4730f51e3/
4220,MAS DE BORRAS,http://krono.act.uji.es/schemas/slodbi#categor...,E
4515,MAS DE BORRAS,http://krono.act.uji.es/schemas/slodbi#ubicacion,node29
4517,MAS DE BORRAS(1),http://krono.act.uji.es/schemas/slodbi#ubicacion,node273
5123,MAS DE BORRAS,http://www.w3.org/2006/vcard/ns#hasAddress,node30
5125,MAS DE BORRAS(1),http://www.w3.org/2006/vcard/ns#hasAddress,node274
6599,MAS DE BORRAS,http://www.w3.org/2006/vcard/ns#hasTelephone,node32


# STEP 2 SPARQL REQUEST GENERATION

In [54]:
sparql = SPARQLWrapper(
    "http://LAPTOP-CS19OKIS:7200/repositories/tach1"
)
sparql.setReturnFormat(JSON)

queryString4="""
SELECT ?o ?p ?s WHERE{
    ?o ?p ?s
    }

"""

In [55]:
sparql.setQuery(queryString4)

test=[]
dataf={}
try:
    ret = sparql.queryAndConvert()     
except Exception as e:
    print(e)

dataf['o']=[]
dataf['p']=[]
dataf['s']=[]
for r in ret["results"]["bindings"]:
    #print(r)
    #print(r["O"]["value"])
    dataf['o'].append(r['o']["value"])
    dataf['p'].append(r['p']["value"])
    dataf['s'].append(r['s']["value"])
    if r['o']["value"] not in test:
        test.append(r['o']["value"])
    if r["s"]["value"] not in test:    
        test.append(r["s"]["value"])    

df=pd.DataFrame(dataf)

automatic dataframe creation for request response

In [56]:
dataframe=pd.DataFrame(ret["results"]["bindings"])
cleandata={}

for l in dataframe.columns:
    cleandata[l]=[]
    

ret["results"]["bindings"]
for r in ret["results"]["bindings"]:
    for l in dataframe.columns:
        cleandata[l].append(r[l]["value"])

dataframe=pd.DataFrame(cleandata)
dataframe

,o,p,s
0,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...
1,http://www.w3.org/2000/01/rdf-schema#subProper...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...
2,http://www.w3.org/2000/01/rdf-schema#subProper...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#TransitiveProperty
3,http://www.w3.org/2000/01/rdf-schema#subClassOf,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...
4,http://www.w3.org/2000/01/rdf-schema#subClassOf,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#TransitiveProperty
...,...,...,...
7562,http://krono.act.uji.es/schemas/slodbi#hotel-7...,http://www.w3.org/2006/vcard/ns#hasEmail,hotelrestaurantelabolera@hotmail.com
7563,http://krono.act.uji.es/schemas/slodbi#hotel-8...,http://www.w3.org/2006/vcard/ns#hasEmail,hotelcrystalpark@gmail.com
7564,http://krono.act.uji.es/schemas/slodbi#hotel-9...,http://www.w3.org/2006/vcard/ns#hasEmail,noucasablanca@gmail.com
7565,http://krono.act.uji.es/schemas/slodbi#hotel-9...,http://www.w3.org/2006/vcard/ns#hasEmail,vinarosplaya@hotelesrh.com


In [57]:
def answer_df(query):

    sparql.setQuery(query)

    try:
        ret = sparql.queryAndConvert()     
    except Exception as e:
        print(e)

    dataframe=pd.DataFrame(ret["results"]["bindings"])
    cleandata={}

    for l in dataframe.columns:
        cleandata[l]=[]
    
    for r in ret["results"]["bindings"]:
        for l in dataframe.columns:
            cleandata[l].append(r[l]["value"])

    dataframe=pd.DataFrame(cleandata)
    return dataframe


In [59]:
bigquery="""
SELECT ?o ?p ?s WHERE{
    ?o ?p ?s
    }
LIMIT 600

"""

#parametric function for query creation following a template 
#possihilities : 
# tiene-hoteles
# tiene-campings
# tiene-albergues
# tiene-empresas-de-turismo-activo

def query_func(qr_variable="?pp",qr_variable1="?s1"):
    if qr_variable1 != "?s1":
        qr_variable1="'"+qr_variable1+"'"
    if qr_variable != "?pp":
        qr_variable="slodbi:"+qr_variable
    
    q="""
    PREFIX slodbi: <http://krono.act.uji.es/schemas/slodbi#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX vcard: <http://www.w3.org/2006/vcard/ns#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    
    SELECT ?s1 ?s3 ?type
    WHERE{
    ?o rdfs:label """+qr_variable1+""".
    ?o """+qr_variable+""" ?s2.
    ?s2 vcard:fn ?s3.
    ?s2 rdf:type ?type.
    }
    
    """
    return q


    
#I want all the hotels  
query=query_func("tiene-campings")   

print(query)


    PREFIX slodbi: <http://krono.act.uji.es/schemas/slodbi#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX vcard: <http://www.w3.org/2006/vcard/ns#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    
    SELECT ?s1 ?s3 ?type
    WHERE{
    ?o rdfs:label ?s1.
    ?o slodbi:tiene-campings ?s2.
    ?s2 vcard:fn ?s3.
    ?s2 rdf:type ?type.
    }
    
    


In [62]:
#automatiser l'ajout des prefixes ? 
doow=answer_df(query)
doow

,s1,s3,type
0,Alcalà de Xivert,LAS MORERAS AC,http://krono.act.uji.es/schemas/slodbi#hotel
1,Alcalà de Xivert,RIBAMAR,http://krono.act.uji.es/schemas/slodbi#hotel
2,Alcalà de Xivert,PLAYA TROPICANA,http://krono.act.uji.es/schemas/slodbi#hotel
3,Alcalà de Xivert,LOS OLIVOS CAMPING PARK,http://krono.act.uji.es/schemas/slodbi#hotel
4,Altura,CAMPING MUNICIPAL,http://krono.act.uji.es/schemas/slodbi#hotel
5,Bejís,LOS CLOTICOS,http://krono.act.uji.es/schemas/slodbi#hotel
6,Benicarló,ALEGRIA DEL MAR,http://krono.act.uji.es/schemas/slodbi#hotel
7,Benicasim,TAURO,http://krono.act.uji.es/schemas/slodbi#hotel
8,Benicasim,AZAHAR,http://krono.act.uji.es/schemas/slodbi#hotel
9,Benicasim,GRAN AVENIDA,http://krono.act.uji.es/schemas/slodbi#hotel


In [63]:
#need to be modified for each request, I don't see how to automate it
#describes which column should be linked to which other column 
edges=[(0,1),(1,2)]        

In [4]:
net = Network("800px","800px",notebook=True)

couleur = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(len(doow.columns))]

#mass=5  pour changer la longueur dee l'arrete

i=0
for col in doow:
    net.add_nodes(doow[col].unique(),title=doow[col].unique(),label=doow[col].unique(),color=[couleur[i]]*doow[col].nunique())
    i=i+1

NameError: name 'doow' is not defined

In [65]:
for m in doow.iterrows():
    for (s,o) in edges:
        net.add_edge(m[1][s],m[1][o])

In [3]:
net.toggle_physics(True)
net.show('mygraph.html')

NameError: name 'net' is not defined

# REST API GET

GraphDB Rest API GET use cases:

list of repositories : http://localhost:7200/rest/repositories  

active repository sql views : http://localhost:7200/rest/sql-views/tables

In [69]:
sparql = SPARQLWrapper(
    "http://LAPTOP-CS19OKIS:7200/repositories/tach1"
)
sparql.setReturnFormat(JSON)


liens=["http://localhost:7200/rest/repositories","http://localhost:7200/rest/sql-views/tables"]
lien=liens[1]
print(lien)

http://localhost:7200/rest/sql-views/tables


3 request methods

In [68]:
x = requests.get(lien)
print(x.content)

b'["Activity","ActivityLocation","CityActivity","ComarcaActivities","Hotels","LINKS","Label","ProvinceLocation","destination inconnue","destinationactive","genereyview"]'


In [70]:
contents = urllib.request.urlopen(lien).read()

contents

b'["Activity","ActivityLocation","CityActivity","ComarcaActivities","Hotels","LINKS","Label","ProvinceLocation","destination inconnue","destinationactive","genereyview"]'

In [71]:
http = urllib3.PoolManager()
r = http.request('GET', lien)
r.data

b'["Activity","ActivityLocation","CityActivity","ComarcaActivities","Hotels","LINKS","Label","ProvinceLocation","destination inconnue","destinationactive","genereyview"]'

# REST API PUT

GraphDB Rest API PUT use cases:

creating a new sql view : http://localhost:7200/rest/sql-views/tables/



In [73]:
viewrqst="SELECT ?s WHERE {?s ?p ?m1. ?m1 ?pp ?s2. \n #!filter \n}"


viewbody={
"name": "viewpython",
"query": viewrqst,
"columns":[
{
"column_name":"s",
"column_type":"string",
"nullable":True,
"sparql_type":"string",
"sql_type_precision":0,
"sql_type_scale":0
}]}

print("body i sent : \n")
print(viewbody)
r = requests.post('http://localhost:7200/rest/sql-views/tables/',json=viewbody)

print("\n response i get : \n")
print(r.text)


body i sent : 

{'name': 'viewpython', 'query': 'SELECT ?s WHERE {?s ?p ?m1. ?m1 ?pp ?s2. \n #!filter \n}', 'columns': [{'column_name': 's', 'column_type': 'string', 'nullable': True, 'sparql_type': 'string', 'sql_type_precision': 0, 'sql_type_scale': 0}]}

 response i get : 

{"message":"Invalid SQL View:  (Not a valid (absolute) IRI: string)."}


GET and POST methods work, we will work on the view creation process in following notebooks